In [ ]:
import cv2
import numpy as np
import os
import shutil

def detect_and_fill_black_areas(image_path, output_path, black_threshold=5, min_area=1000, merge_distance=10, inpaint_radius=3):
    # Загрузка изображения
    image = cv2.imread(image_path)
    if image is None:
        print(f"Ошибка: Не удалось загрузить изображение {image_path}. Проверьте целостность файла.")
        return False

    # Преобразование изображения в grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Бинаризация изображения для выделения черных областей
    _, binary = cv2.threshold(gray, black_threshold, 255, cv2.THRESH_BINARY_INV)

    # Поиск контуров
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Фильтрация контуров по площади
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]

    # Если контуров нет, копируем файл в папку обработанных файлов
    if not filtered_contours:
        print(f"Не найдено подходящих черных областей в {image_path}. Файл будет скопирован.")
        shutil.copy(image_path, output_path)
        return False

    # Объединение близких контуров
    merged_contours = []
    for contour in filtered_contours:
        x, y, w, h = cv2.boundingRect(contour)
        merged = False
        for i, merged_contour in enumerate(merged_contours):
            mx, my, mw, mh = cv2.boundingRect(merged_contour)
            # Если текущий контур близок к уже объединенному, объединяем их
            if (abs(x - mx) < merge_distance and abs(y - my) < merge_distance):
                merged_contours[i] = np.vstack((merged_contour, contour))
                merged = True
                break
        if not merged:
            merged_contours.append(contour)

    # Создание маски для заливки
    mask = np.zeros_like(gray)
    for contour in merged_contours:
        cv2.drawContours(mask, [contour], -1, 255, -1)  # Заливаем контуры белым цветом

    # Определение доминирующего цвета для каждой области
    for contour in merged_contours:
        # Получаем ограничивающий прямоугольник для контура
        x, y, w, h = cv2.boundingRect(contour)

        # Расширяем область для анализа окружения
        expand_size = 10  # Размер расширения для анализа окружения
        x1 = max(x - expand_size, 0)
        y1 = max(y - expand_size, 0)
        x2 = min(x + w + expand_size, image.shape[1])
        y2 = min(y + h + expand_size, image.shape[0])

        # Вырезаем область для анализа
        roi = image[y1:y2, x1:x2]

        # Определяем доминирующий цвет в окружении
        dominant_color = np.array(cv2.mean(roi)[:3], dtype=np.uint8)

        # Заливаем область доминирующим цветом
        cv2.drawContours(image, [contour], -1, dominant_color.tolist(), -1)

    # Сглаживание границ с помощью inpainting
    image = cv2.inpaint(image, mask, inpaintRadius=inpaint_radius, flags=cv2.INPAINT_TELEA)

    # Сохранение результата
    cv2.imwrite(output_path, image)
    print(f"Изображение сохранено как {output_path}")
    return True

def process_folder(input_folder, output_folder):
    # Проверка существования папок
    if not os.path.exists(input_folder):
        print(f"Ошибка: Папка '{input_folder}' не найдена.")
        return

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Обработка всех изображений в папке
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)
            detect_and_fill_black_areas(input_path, output_path)

# Пример использования
input_folder = 'blood_analysis_dataset_540/paper'
output_folder = 'blood_analysis_dataset_540/processed'
process_folder(input_folder, output_folder)